In [53]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
import numpy as np

In [54]:
def get_word_probability(logits: torch.tensor, word_index: int) -> torch.float:
    

In [55]:
def get_logits_for_masked_sentence(sentence: str):
    inputs = tokenizer(sentence, return_tensors='pt')
    outputs = model(**inputs)
    return outputs.logits

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

In [79]:
def score_sentence(sentence: str) -> float:
    
    masked_words_probs = []
    
    sentence_words = sentence.split()
    for index in range(len(sentence_words)):
        masked_word = sentence_words[index]
        masked_sentence = (' '.join(sentence_words[:index]) + " [MASK] " + ' '.join(sentence_words[index + 1:])).strip()
        
        inputs = tokenizer(masked_sentence, return_tensors='pt')
        outputs = model(**inputs)
        
        logits = outputs.logits
        
        index = np.where(inputs.input_ids[0] == tokenizer.mask_token_id)[0][0]
        
        masked_word_logit = logits[0, index, tokenizer.convert_tokens_to_ids([masked_word])[0]]
        masked_word_prob = torch.sigmoid(masked_word_logit)
        
        masked_words_probs.append(masked_word_prob.detach().numpy())
        
    return -np.sum(np.log(masked_words_probs))

sentence = "The capital of Spain is Madrid"
score_sentence(sentence)

4.6793594